This notebook is designed to demonstrate how to produce a range of plots for airborne EM (line data) and SNMR (point data) on a vertical section. The requirements here are for the AEM inversion data to be in netcdf format as generated by the geophys_utils netcdf conversion tool and for the SNMR data to be in a sptialite database.

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib inline

In [2]:
import netCDF4
from geophys_utils._netcdf_line_utils import NetCDFLineUtils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
from hydrogeol_utils.plotting_utils import ConductivitySectionPlot
import time
import os

C:\Users\symin\Anaconda3\envs\hydrogeol_utils\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# First we will deomstrate some straight AEM plotting

# Open netcdf files

ncdir = r'C:\GA\OB_AEM\inversions\netcdf\\'

# Open the file with the EM measurements
# Here we use the data response file provided by Niel Christensen
EM_path = ncdir + 'OrdBon2019_DataResp_cord2dLogKeep.nc'
EM_dataset = netCDF4.Dataset(EM_path)


# Open the file with the conductivity values
# The conductivity model was a 2d correlated borehole
# constrained inversion done by Niel Christensen

cond_path = ncdir + 'OrdBon2019_ModelExp_cord2dLogKeep.nc'
cond_dataset = netCDF4.Dataset(cond_path)

In [4]:
# View datasets

print(EM_dataset)
print(cond_dataset)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Dataset read from ASEG-GDF file OrdBon2019_DataResp_cord2dLogKeep.dat
    Conventions: CF-1.6,ACDD-1.3
    featureType: trajectory
    geospatial_vertical_min: -6.44
    geospatial_vertical_max: 183.83
    geospatial_vertical_units: m
    geospatial_vertical_resolution: point
    geospatial_vertical_positive: up
    history: Converted from ASEG-GDF file OrdBon2019_DataResp_cord2dLogKeep.dat using definitions file OrdBon2019_DataResp_cord2dLogKeep.dfn
    date_created: 2018-12-03T10:29:47.069712
    geospatial_east_resolution: point
    geospatial_north_resolution: point
    geospatial_east_min: 439425.0
    geospatial_east_max: 519593.2
    geospatial_east_units: m
    geospatial_north_min: 8272445.8
    geospatial_north_max: 8371327.5
    geospatial_north_units: m
    geospatial_bounds: POLYGON((497426.3000 8272445.8000, 462013.2000 8307557.2000, 458094.8000 8312039.8000, 457222.0000 83143

In [5]:
# airborne EM inversions and data are typically visualised on a line by line basis

# To utilise the geophys_utils for line data create a NetCDFLineUtils for each dataset

EM_line_utils = NetCDFLineUtils(EM_dataset)
cond_line_utils = NetCDFLineUtils(cond_dataset)

# Display the lines for the conductivity mode

lines = cond_line_utils.line
print(lines)

[100102 100202 100302 100401 100501 100601 100701 100801 100901 101001
 101102 101202 101301 101402 101502 101602 101702 101801 101901 102001
 102101 102102 102201 102301 102401 102501 102601 102701 102801 102802
 102901 103001 103101 103201 103301 103302 103401 103501 103601 103701
 103801 103901 104001 104101 104201 104301 104401 104501 104601 104701
 104801 104901 105001 105101 105201 105301 105401 105501 105601 105701
 105801 105901 106001 106101 106201 106301 106401 106501 106601 106701
 106801 106901 107001 107101 107201 107301 107401 107501 107601 107701
 107702 107801 107901 108001 108101 108201 108301 108401 108501 108502
 108601 108701 108801 108901 108902 109001 109101 109201 109301 109302
 109401 109501 109601 500102 500201 500301 500401 500501 500601 912001
 912002 912003 912004 912005 912006 912007]


In [6]:
# Create an instance of plots
plots = ConductivitySectionPlot(cond_dataset, EM_dataset)

# Define some key variables which we want to inteprolate
section_vars = {'conductivity_model': ['conductivity', 'data_residual', 'tx_height_measured', 'depth_of_investigation'],
                 'EM_data': ['data_values_by_low_moment_gate', 'data_values_by_high_moment_gate', 
                 'data_response_by_low_moment_gate', 'data_response_by_high_moment_gate']}

# Define the hdf5 output directory
hdf5_dir =  r"C:\GA\OB_AEM\inversions\OrdBon2019_grids_hdf5"

# Define the resolution of th sections
xres, yres = 20., 2.

# Here we are plotting every line and saving the outputs as hdf5 files
# Given some of the computing time overhead of gridding the sections it may be preferrable to save
# the output to allow faster plotting
plots.grid_vars(xres = xres, yres =yres, lines=lines, conductivity_variables = section_vars['conductivity_model'],
                  data_variables = section_vars['EM_data'],
                     layer_subdivisions = 4, resampling_method = 'linear',
                     save_hdf5 = True, hdf5_dir = hdf5_dir,
                     overwrite_hdf5 = True, return_dict = False)


0.0


In [7]:
# Now we will plot a section from one of the hdf5 file we produced earlier

line = 104201

# The sections will be plotted with a number of panels with a shared horizontal axis
# To define what we will plot for each panel we use a dictionary that define the variable
# to plot (which needs to be inluded in the interpolated) and the 

plot_settings = {'figsize': (20, 11), 'dpi': 300}

panel_settings = {'panel_1': {'variable': 'data_values_by_high_moment_gate',
                              'plot_type': 'multi_line',
                             'panel_kwargs': {'title': 'high moment data',
                                             'ylabel': 'dB/dT (V/(A.turns.m^4))'},
                             'height_ratio': 3},
                  
                 'panel_2': {'variable': 'data_values_by_low_moment_gate',
                             'plot_type': 'multi_line',
                             'panel_kwargs': {'title': 'low moment data',
                                              'ylabel': 'dB/dT (V/(A.turns.m^4))'},
                             'height_ratio': 3},
                  
                 'panel_3': {'variable': 'data_residual',
                             'plot_type': 'line',
                             'panel_kwargs': {'title': 'data residual', 'color': 'black',
                                              'ylabel': 'data residual',
                                              'legend': False},
                             'height_ratio': 1},
                  
                 'panel_4': {'variable': 'conductivity',
                             'plot_type': 'grid',
                             'panel_kwargs': {'title': 'AEM conductivity',
                                              'max_depth': 400, 'shade_doi': True, 'colourbar': True,
                                              'colourbar_label': 'Conductivity (S/m)',
                                             'log_plot': False, 'vmin': 0, 'vmax': 0.1,
                                             'cmap': 'jet', 'ylabel': 'elevation_(mAHD)'},
                             'height_ratio': 5},
                  
                 'panel_5': {'variable': 'conductivity',
                             'plot_type': 'grid',
                             'panel_kwargs': {'title': 'AEM conductivity',
                                              'max_depth': 400, 'shade_doi': True, 'colourbar': True,
                                              'colourbar_label': 'Conductivity (S/m)',
                                             'log_plot': True, 'vmin': 0.001, 'vmax': 0.1,
                                            'cmap': 'jet', 'ylabel': 'elevation_(mAHD)'},
                             'height_ratio': 5}}
                
# Extract the data set from the path
hdf5_dir =  r"C:\GA\OB_AEM\inversions\OrdBon2019_grids_hdf5"
file = os.path.join(hdf5_dir, str(line) + '.hdf5')

# We will save this plot to a directory

outdir = r'C:\GA\OB_AEM\inversions\sections_cor2dlog'

fig, ax_array = plt.subplots(len(panel_settings), 1, sharex=True, figsize=(20, 11),
                                     gridspec_kw={'height_ratios': plots.unpack_plot_settings(panel_settings,
                                                                                             'height_ratio')})

plots.plot_conductivity_section_from_hdf5file(file, ax_array, plot_settings, panel_settings, save_fig = True,
                                  outfile = os.path.join(outdir, str(line) + '.png'))


In [22]:
# Otherwise plotting can be done automatically stright from a gridded dictionary
# if no hdf5 file exists

# We will use the same panel settings

line = 105501

dat_vars = ['data_values_by_high_moment_gate', 'data_values_by_low_moment_gate']
cond_vars = ['data_residual', 'tx_height_measured', 'depth_of_investigation',
             'conductivity']
# Grid the line but leave it in memory
gridded_vars = plots.grid_vars(xres = 20., yres =4., lines=line,
                               conductivity_variables = cond_vars,
                               data_variables = dat_vars,
                               layer_subdivisions = 4, resampling_method = 'linear',
                               save_hdf5 = False, return_dict = True)

fig, ax_array = plt.subplots(len(panel_settings), 1, sharex=True, figsize=(20, 11),
                                     gridspec_kw={'height_ratios': plots.unpack_plot_settings(panel_settings,
                                                                                             'height_ratio')})

# Now plot
plots.plot_conductivity_section(ax_array, gridded_vars[line], plot_settings, panel_settings, save_fig = False,
                                  outfile = os.path.join(outdir, str(line) + '.png'), return_axes=True)

In [ ]:
# Next we want to add other data over the conductivity sections

###TODO : add SNMR stick models